# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
24/04/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import cv2
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x000001EF8826A330>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [3]:
# SENSIBILITE CAPTEUR
# S_force_bs =       # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

# 30/05/2024

In [4]:
# LOAD DATAs

# matrice vide
data = []
pentes = []

# labels 
anches = ['Force 3 1/2']
mymap = plt.get_cmap("Spectral")

# load data
data = pd.read_table('2024.05.30/data_anche1.txt', sep=' ', header=1).values # contient : [y, z, offset, moyenne_brute, moyenne(N) en grandeur physique]
position_y = data[:,0]
position_z = data[:,1]
forces = data[:,4]

In [5]:
ys, _ = np.unique(position_y, return_counts=True)
zs, _ = np.unique(position_z, return_counts=True)


plot_pentes = True
if plot_pentes:  
    plt.figure()
pentes = []
for idx, m in enumerate(ys) :  
    couleur = mymap(m / len(ys))
    y = position_y == m
    z = position_z[y]
    F = forces[y]
    
    deb = 5
    coefficients= np.polyfit(z[deb:], F[deb:], 1)   # curve fitting par ajustement polynomial d'ordre 1
    pente = coefficients[0]
    pente = round(pente, 2)
    pentes.append(pente)
    if plot_pentes:
        plt.plot(z, F, '.-', label=f'{m} mm')
        plt.plot(z[deb:], np.polyval(coefficients, z[deb:]), linestyle='--', color=couleur, label=f'{pentes[idx]}')
    
if plot_pentes:   
    plt.legend(title = 'Position sur largeur d\'anche')
    plt.xlabel('Coordonnée z (mm)', fontsize=16)
    plt.ylabel('Force (N)', fontsize=16)
    plt.grid()


model_v1 = True  
if model_v1 :
    # réajustement du format des données
    pos_y = np.array(ys).reshape(-1,1)
    pentes = np.array(pentes)
    # curve fitting de 2nd ordre
    poly = PolynomialFeatures(degree=2)
    poly_y = poly.fit_transform(pos_y)
    # regression linéaire
    reg_model = LinearRegression()
    reg_model.fit(poly_y, pentes)
    coef = reg_model.coef_
    intercept = reg_model.intercept_
    # prédiction de la parabole
    pentes_pred = reg_model.predict(poly_y)
    R2 = reg_model.score(poly_y, pentes)
    # plot
    plt.figure(figsize=(10, 6))
    plt.plot(ys, pentes, '.-')
    plt.plot(ys, pentes_pred, linestyle='--', label=f"y = {coef[2]:.2f} * x² + {coef[1]:.2f} * x + {intercept:.2f} \n R² = {round(R2,3)}")
    plt.xlabel('Coordonnée y (mm)', fontsize=20)
    plt.ylabel('Raideur $k=F/z$ (N/mm)', fontsize=20)
    plt.legend(fontsize=15)
    plt.grid()

    
model_v2 = True  
if model_v2: 
    coef = np.polyfit(ys, pentes, 2)
    R2 = reg_model.score(poly_y, pentes)
    # R2 = skl.metrics.r2_score(pentes, np.polyval(pentes, ys))
    plt.figure()
    plt.plot(ys, pentes, '.-')
    plt.plot(ys, np.polyval(coef, ys), linestyle='--', label=f'y = {coef[0]:.2f}x² + {coef[1]:.2f}x + {coef[2]:.2f} \nR² = {round(R2,2)}')
    plt.xlabel('Coordonnée y (mm)', fontsize=16)
    plt.ylabel('Raideur $k=F/z$ (N/mm)', fontsize=16)
    plt.legend()
    plt.grid()


In [35]:
# Faire une fonction qui vient répertorier les pentes pour tous un cas random
# y : tableau des positions de y
# z
# F :  tableau des forces
# y, z, F sont issus des datas du .txt aqui par python 

def pentes(position_y, position_z, Force): # a galère c'est de prendre la force de ce qui nous intéresse donc d'appliquer le même masque sur la force avant
    pentes = [] # vecteur vide pour receuillir les pentes
    deb = np.where(Force > 0.2*max(Force))[0][0] # indice du début de la zone d'intérêt : pour être certain.e que l'on est dans la partie linéaire, qu'on considère comme 20% de la valeur max
    
    for m in (np.unique(position_y, return_counts=True)[0]) :  # pentes pour chaque position position y
        y  = position_y == m 
        z = position_z[y]
        F = Force[y]
        # print(Force)
        
        coefficients= np.polyfit(z[deb:], F[deb:], 1)   # curve fitting par ajustement polynomial d'ordre 1
        pente = coefficients[0]
        pente = round(pente, 2)
        pentes.append(pente)
        
        # coef = LinearRegression().fit(poly_y, pentes).coef_
    print(pentes)
    return pentes



# faire un truc pour avoir le modèle et la valeur de R²

In [36]:
data = pd.read_table('2024.05.30/data_anche1.txt', sep=' ', header=1).values # contient : [y, z, offset, moyenne_brute, moyenne(N) en grandeur physique]
position_y = data[:,0]
position_z = data[:,1]
forces = data[:,4]

ptss = pentes(position_y, position_z, forces)

[1.16, 1.58, 1.96, 2.3, 2.53, 2.64, 2.76, 2.67, 2.58, 2.41, 2.06, 1.66, 1.2]


In [47]:
# nombre de points sur la largeur de l'anche
ys, _ = np.unique(position_y, return_counts=True)
nbr_y = ys.shape[0]


ys_13 = ys
ys_11 = [ys[i] for i in [0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12]]
ys_9 = [ys[i] for i in [0, 1, 3, 4, 6, 8, 9, 11, 12]]
ys_7 = ys[::2]
ys_5 = ys[::3]

Y = [ys_13, ys_11, ys_9, ys_7, ys_5]

for y in Y:
    mask = np.isin(position_y, y)
    pos_y = np.array(position_y[mask])
    pos_z = np.array(position_z[mask])
    f = np.array(forces[mask])
    
    pentes = pentes(pos_y, pos_z, f)
    plt.figure()
    plt.plot(np.array(y), pentes, '.-')
    # plt.plot(position_y, np.polyval(coef, position_y), linestyle='--', label=f'y = {coef[0]:.2f}x² + {coef[1]:.2f}x + {coef[2]:.2f} \nR² = {round(R2,2)}')
    plt.xlabel('Coordonnée y (mm)', fontsize=16)
    plt.ylabel('Raideur $k=F/z$ (N/mm)', fontsize=16)
    plt.legend()
    plt.grid()    

affichage = False
if affichage:
    print('ys_13 =', ys_13)
    print('ys_11 =', ys_11)
    print('ys_9 =', ys_9)
    print('ys_7 =', ys_7)
    print('ys_5 =', ys_5)
    print('Y =', Y)


TypeError: 'list' object is not callable

In [32]:
# nombre de points sur la largeur de l'anche
zs, _ = np.unique(position_z, return_counts=True)
nbr_z = zs.shape[0]
print(nbr_z)


zs_21 = zs
zs_18 = [zs[i] for i in [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20]]
zs_16 = [zs[i] for i in [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20]]
zs_14= [zs[i] for i in [0, 1, 3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19]]
zs_11 = zs[::2] 
zs_7 = zs[::3]

Z = [zs_21, zs_18, zs_16, zs_14, zs_11, zs_7]

affichage = False
if affichage:
    print('zs_21 =', zs_21)
    print('zs_18 =', len(zs_18))
    print('zs_16 =', len(zs_16))
    print('zs_14 =',len( zs_14))
    print('zs_11 =',len( zs_11))
    print('zs_7 =',len( zs_7))
    print('Z =', len(Z))


21


In [ ]:
# faire un boucle qui répertorie les cas pour tracer les parabole avec le modèle et qui vient récupere le R² (root mean square)

# for y in (Y):
#     for Z in (Z):
#         plt.plot()
        
# tracer l'évolution de R² selon y et z

# 16.05.2024

In [17]:
plt.close('all')

# matrice vide
data = []
incertitudes = []
ecarts_types = []


# labels 
anches = ['Force 3 1/2']
position_y = [0, 2.5, 5]
position_z = [0.3, 0.6, 0.9, 1.2]

# palette de couleurs
palette_couleurs = plt.cm.viridis

# load data
data = pd.read_table(f'repetabilite_x_4mm_0/Anche{0}.txt', sep=' ', header=1).values

plot_rep_z = True
if plot_rep_z:
    plt.figure(f'Répétabilité z')
    plt.title(f'Répétabilité z')
    # repetabilite pour 3 positions sur la largeur d anche
    for m in range (3) :
        couleur = palette_couleurs(m/3)
        means =  []
        ecarts_types = []
        proportions = []
        if (data[:, 0] == m).any():
            # repetabilite pour 4 positions sur la profondeur
            for n in range(4):
                if (data[:, 1] == n).any():
                    subset = data[(data[:, 0] == m) & (data[:, 1] == n)]
                    if subset.size > 0:  # Assurez-vous que le sous-ensemble n'est pas vide
                        rep = data[data[:, 0] == m, 2]
                        force = subset[:, 5]
                        mean = np.mean(force)
                        means.append(mean)
                        ecart_type = np.std(force, ddof=1)
                        proportion = ecart_type/mean
                        ecarts_types.append(ecart_type)
                        proportions.append(proportion*100)
            coefficients = np.polyfit(position_z, means, 1)
            pente = round(coefficients[0], 2)
            print('pente', pente)
            print(f'erreur en % pour la position y={position_y[m]} : {proportions}')
            plt.plot(position_z, means, '.-', label=f'{position_y[m]} mm', color = couleur)
            plt.errorbar(position_z, means, yerr=ecarts_types, fmt='.', color=couleur, capsize=5) 
            plt.xlabel('Position z - profondeur d\'appui (mm)')
            plt.ylabel('Force (N)')
            plt.legend(title = 'Position y')
            plt.grid(True)
            
        
plot_rep_y = False
if plot_rep_y:
    plt.figure(f'Répétabilité y')
    plt.title(f'Répétabilité y')
    # repetabilite pour 4 positions sur la profondeur 
    for m in range (4) :
        couleur = palette_couleurs(m/3)
        means =  []
        ecarts_types = []
        if (data[:, 1] == m).any():
            # repetabilite pour 3 positions sur la largeur d anche
            for n in range(3):
                if (data[:, 1] == n).any():
                    subset = data[(data[:, 1] == m) & (data[:, 0] == n)]
                    if subset.size > 0:  # Assurez-vous que le sous-ensemble n'est pas vide
                        rep = data[data[:, 0] == m, 2]
                        force = subset[:, 5]
                        mean = np.mean(force)
                        means.append(mean)
                        # print(means)
                        ecart_type = np.std(force, ddof=1)
                        ecarts_types.append(ecart_type)
                        proportion = ecart_type/mean
                        print('proportion =',proportion*100)
            
            coefficients = np.polyfit(position_y, means, 1)
            pente = round(coefficients[0], 2)
            print('pente', pente)
            plt.plot(position_y, means, '.-', label=f'{position_z[m]} mm', color = couleur)
            plt.errorbar(position_y, means, yerr=ecarts_types, fmt='.', color=couleur, capsize=5) 
            plt.xlabel('Position y sur la largeur d\'anche (mm) \n- milieu anche = 0')
            plt.ylabel('Force (N)')
            plt.legend(title = 'profondeur appui')
            plt.grid(True)



plt.show()

pente 2.71
erreur en % pour la position y=0 : [1.4598633461303907, 0.881899878966979, 0.8339585336248552, 0.5442206038994835]
pente 2.43
erreur en % pour la position y=2.5 : [2.741354331086734, 0.8637514384784795, 0.6487342005030842, 0.6301104310880056]
pente 1.52
erreur en % pour la position y=5 : [7.830145780801861, 1.710421171058678, 0.9537303176982805, 0.6577151461635921]


### Incertitude sur pente

In [6]:

# matrice vide
data = []
incertitudes = []
ecarts_types = []

# labels 
anches = ['Force 3 1/2']
position_y = [0, 2.5, 5]
position_z = [0.3, 0.6, 0.9, 1.2]

# palette de couleurs
palette_couleurs = plt.cm.viridis

# load data
data = pd.read_table(f'repetabilite_x_4mm_0/Anche{0}.txt', sep=' ', header=1).values

plot_rep_z = True
if plot_rep_z:
    plt.figure(f'Répétabilité z')
    plt.title(f'Répétabilité z')
    # repetabilite pour 3 positions sur la largeur d anche
    for m in range (4) :
        couleur = palette_couleurs(m/3)
        means =  []
        ecarts_types = []
        if (data[:, 0] == m).any():
            # repetabilite pour 4 positions sur la profondeur
            for n in range(4):
                if (data[:, 1] == n).any():
                    subset = data[(data[:, 0] == m) & (data[:, 1] == n)]
                    if subset.size > 0:  # Assurez-vous que le sous-ensemble n'est pas vide
                        rep = data[data[:, 0] == m, 2]
                        force = subset[:, 5]
                        mean = np.mean(force)
                        means.append(mean)
                        ecart_type = np.std(force, ddof=1)
                        ecarts_types.append(ecart_type)
            # erreur sur la pente
            slope, intercept, r_value, p_value, std_err = sp.stats.linregress(position_z, means)
            print(slope, intercept, r_value, p_value, std_err)
            
            plt.plot(position_z, means, '.-', label=f'{position_y[m]} mm', color = couleur)
            plt.errorbar(position_z, means, yerr=ecarts_types, fmt='.', color=couleur, capsize=5) 
            plt.xlabel('Position z - profondeur d\'appui (mm)')
            plt.ylabel('Force (N)')
            plt.legend(title = 'Position y')
            plt.grid(True)

2.7122149999999996 -0.22075249999999946 0.9999226758329222 7.732416707784129e-05 0.023851018322113993
2.4270799999999997 -0.27534999999999976 0.9999491949645973 5.0805035402667884e-05 0.017300315347177585
1.5198983333333334 -0.2782374999999999 0.9999910121308714 8.987869128618273e-06 0.004556651731276993


### Incertitudes sur pente : pente à pente

In [7]:

plt.close('all')

# matrice vide
data = []
incertitudes = []
ecarts_types = []


# labels 
anches = ['Force 3 1/2']
positions_y = [0, 2.5, 5]
positions_z = [0.3, 0.6, 0.9, 1.2]

# palette de couleurs
palette_couleurs = plt.cm.viridis


# test de répétabilité effectue sur 4 anches différentes
# for i in range (1):
    # couleur = palette_couleurs(i/4)
    # recuperation donnees
data = pd.read_table(f'repetabilite_x_4mm_0/Anche{0}.txt', sep=' ', header=1).values

plt.figure(f'Répétabilité z')
plt.title(f'Répétabilité z')
# repetabilite pour 3 positions sur la largeur d anche
for m in range (3) :
    pentes = []
    couleur = palette_couleurs(m/3)
    if (data[:, 0] == m).any():
        # 20 répétitions
        for n in range(20):
            forces = []
            if (data[:, 2] == n).any():
                # pour 4 positions sur la profondeur
                for o in range(4):
                    subset = data[(data[:, 0] == m) & (data[:, 2] == n)]
                    if subset.size > 0:  # Assurez-vous que le sous-ensemble n'est pas vide
                        position_z = data[data[:, 0] == m, 1]
                        force = subset[:, 5]
                        forces.append(force)
                        # print(len(forces))
                plt.plot(positions_z, forces[0], '.-', label=f'{position_y[m]} mm', color = couleur)
                
                coeffs = np.polyfit(positions_z, forces[0], 1)
                pente = coeffs[0]
                # print(pente)
                # print(len(pentes))
                pentes.append(pente)
                force_pred = np.polyval(coeffs, positions_z)
                # plt.plot(positions_z, force_pred, color='red')
        sigma_pente  = np.std(pentes, ddof=1)
        print(sigma_pente*100)
                
                        
                    
        plt.xlabel('Position z - profondeur d\'appui (mm)')
        plt.ylabel('Force (N)')
        # plt.legend(title = 'Position y')
        plt.grid(True)
        



plt.show()

1.9528765632638045
2.7370916395464095
1.7693073528282037


In [18]:
plt.close('all')

# matrice vide
data = []
incertitudes = []
ecarts_types = []


# labels 
anches = ['Force 3 1/2']
position_y = [0, 2.5, 5]
position_z = [0.3, 0.6, 0.9, 1.2, 0.9, 0.6, 0.3]

# palette de couleurs
palette_couleurs = plt.cm.viridis


data = pd.read_table(f'rep_hyste_x_4mm_0/Anche{0}.txt', sep=' ', header=1).values
# print(data)

plt.figure(f'Hysteresis')
plt.title(f'Hysteresis')
# repetabilite pour 3 positions sur la largeur d anche
for m in range (4) :
    couleur = palette_couleurs(m/3)
    means =  []
    ecarts_types = []
    if (data[:, 0] == m).any():
        # repetabilite pour 4 positions sur la profondeur
        for n in range(7):
            if (data[:, 1] == n).any():
                subset = data[(data[:, 0] == m) & (data[:, 1] == n)]
                if subset.size > 0:  # Assurez-vous que le sous-ensemble n'est pas vide
                    rep = data[data[:, 0] == m, 2]
                    force = subset[:, 5]*29/285
                    mean = np.mean(force)
                    means.append(mean)
                    # print(means)
                    ecart_type = np.std(force, ddof=1)
                    proportion = ecart_type/mean
                    # print('proportion =',proportion*100)
                    ecarts_types.append(ecart_type)

        
        plt.plot(position_z, means, '.-', label=f'{position_y[m]} mm', color = couleur)
        for i in range(len(position_z) - 1):
            plt.annotate('', xy=((position_z[i]+position_z[i+1])/2, (means[i]+means[i+1])/2), xytext=(position_z[i], means[i]), arrowprops=dict(arrowstyle='->', color=couleur))
    
        plt.errorbar(position_z, means, yerr=ecarts_types, fmt='.', color=couleur, capsize=5) 
        plt.xlabel('Position z - profondeur d\'appui (mm)')
        plt.ylabel('Force (N)')
        plt.legend(title = 'Position y')
        plt.grid(True)

# 17.04.2024

In [9]:
# SENSIBILITE CAPTEUR
S_force_bs = 29.67      # bs pour banc statique, sensibilité en N/mV
G_force_bs = 100        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

## Traitement données

In [19]:
plt.close('all')

# matrice vide
data = []
incertitudes = []
ecarts_types = []


# labels 
anches = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
positions = [0.2, 0.4, 0.6]

# palette de couleurs
palette_couleurs = plt.cm.viridis

plt.figure(f'Répétabilité')
plt.title(f'Répétabilité')

# test de répétabilité effectue sur 4 anches différentes
for i in range (4):
    couleur = palette_couleurs(i/4)
    # recuperation donnees
    data = pd.read_table(f'2024.04.17-repetabilite/repetabilite_{i+1}/Anche{i+1}.txt', sep=' ', header=1).values
    means =  []

    # repetabilite pour 3 positions sur la largeur d anche
    for m in range (3) :
        
        if (data[:, 0] == m).any() :
            rep = data[data[:, 0] == m, 1]
            force = data[data[:, 0] == m, 4]
        mean = np.mean(force)
        means.append(mean)
        ecart_type = np.std(force, ddof=1)
        # LA Y A DU TRAVAIL -> critère de validation du 
        ecarts_types.append(ecart_type)
        print(ecart_type/mean)

    plt.plot(positions, means, '.-', label=f'{anches[i]}', color = couleur)
    plt.errorbar(positions, means, yerr=ecart_type, fmt='.', color=couleur, capsize=5)    
    coefficients = np.polyfit(positions, means, 1)       # courbe paramétrique 
    polynome = np.poly1d(coefficients) 
    x = np.linspace(min(positions), max(positions), 100)
    y = polynome(x)
    print('pente', round(coefficients[0], 2))
    # plt.plot(x, y, color=couleur, label=f'Courbe paramétrique ajustée de pente {coefficients[0]}')
plt.xlabel('Position y (mm)')
plt.ylabel('Force (N)')
plt.legend(title = 'Numéro d\'anche')
plt.grid(True) 


plt.show()

0.2764426127618193
0.06347438617973093
0.07576148494710938
pente 1.4
0.22156575567688072
0.03838290892399765
0.03960554607872783
pente 2.24
0.10439084873775939
0.04850847925296376
0.020805196517594073
pente 2.74
0.10612427239034218
0.038648587490650255
0.024761308742099364
pente 3.78


In [20]:
plt.close('all')

# matrice vide
data = []
A = []

# labels 
anches = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
force_appui = [0.5, 0.7, 0.9, 1.1]

# palette de couleurs
palette_couleurs = plt.cm.viridis


# profil d'anche
for i in range (4):
    plt.figure(f'Profil d\'anche num {i+1}')
    plt.title(f'Profil d\'anche n°{i+1} ({anches[i]})')
    for j in range (4):
        couleur = palette_couleurs(j/4)
        # recuperation donnees
        # data = pd.read_table(f'Banc statique/2024.04.17-profil_anche/profil_{i+1}_{j+1}/data{j+1}.txt', sep=' ', header=1).values
        data = pd.read_table(f'2024.04.17-profil_anche/profil_{i+1}_{j+1}/data{j+1}.txt', sep=' ', header=1).values
        # print(data)
        position = data[:,0]
        force = data[:,2]
        plt.plot(position, force, '.-', label=f'position d\'appui {force_appui[j]} mm', color = couleur)
        
        coefficients = np.polyfit(position, force, 4)       # courbe paramétrique 
        polynome = np.poly1d(coefficients)
        x = np.linspace(min(position), max(position), 100)
        y = polynome(x)
        # print(coefficients)
        # plt.plot(x, y, color=couleur, label='Courbe paramétrique ajustée')
        # plt.errorbar(position, force, yerr=ecart_type_tot, fmt='.', color=couleur, capsize=5)  # barres d'erreurs
        plt.xlabel('y (mm)')
        plt.ylabel('Force (N)')
        plt.legend()
        plt.grid(True)
        

plt.show()

In [21]:
# graphique 2D de la force appliquée en chaque point de mesure
plt.close('all')

import matplotlib.colors as mcolors


# dossier = "Banc statique/2024.04.23/" # ordi pro
dossier = "2024.04.23/"
savename = "pentes"
ys = []
pentes = []
mymap = plt.get_cmap("Spectral")
# mymap = plt.get_cmap("twilight")

Forces = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
n = len(Forces)

for j in range (n):
    plt.figure(f'Anche {j+1}')
    plt.title(f'Force mesurée en fonction de l\'appui vertical (position en z) pour chaque \n position y sur la largeur d\'anche - Anche n°{j+1} ({Forces[j]})', fontsize=16)
    # recuperation donnees
    data = pd.read_table(dossier + f'anche{j+1}/data_anche{j+1}.txt', sep=' ', header=0).values 
    indices_tries = np.lexsort((data[:, 1], data[:, 0]))
    data = data[indices_tries] 
    # print(data)
    y = data[:,0]
    z = data[:,1]
    force = data[:,2]
    N = 6
    a=1
    for i in range (0, len(data), N):
        # palette de couleurs
        couleur = mymap(i / len(data))
        
        # trace les courbes de la force mesurées en fonction de l'appuie pour chaque position sur la largeur d'anche
        plt.plot(z[i:i+N], force[i:i+N], marker='*', color=couleur, label=f'{np.round(y[i], 2)} mm')
        
        # extractions des paramètres de la courbe paramétrique
        coefficients= np.polyfit(z[i+a:i+N], force[i+a:i+N], 1)
        pente = coefficients[0]
        pente = round(pente, 2)
        ys.append(y[i])
        pentes.append(pente)
        # pentes.append(['anche', j, 'position y', y[i], 'pente', pente])
        
        # trace la courbe paramétrique sur la section qui nous intéresse
        # plt.plot(x=data_a[i:i+4,1], np.polyval(coefficients, data_a[i:i+4,1]), label=f'Droite de régression (pente={pente:.2f}, ordonnée origine={ordonnee_origine:.2f})')
        plt.plot(z[i+a:i+N], np.polyval(coefficients, z[i+a:i+N]), linestyle='--', color=couleur, label=f'pente={pente:.2f}')
        # plt.errorbar(z[i:i+N], force[i:i+N], yerr=ecart_type, fmt='.', color=couleur, capsize=5)

        # bhznjf,ok
        R = np.corrcoef(force[i+2:i+N], np.polyval(coefficients, z[i+2:i+N]))
        # print(round(R[1,0],4))



        plt.xlabel('Coordonnée z (mm)', fontsize=16)
        plt.ylabel('Force (N)', fontsize=16)
        plt.grid(True)
        plt.legend(title='position d\'appui')
        
    
# print(f'pente n°{i} = {pentes} N/mm')
    # enregister en .txt
    # np.savetxt(dossier + savename, 'pentes')
# print(len(pentes))
# print(len(ys))
plt.show()

In [13]:
# graphique 2D de la force appliquée en chaque point de mesure
plt.close('all')

import matplotlib.colors as mcolors


# dossier = "Banc statique/2024.04.23/" # ordi pro
dossier = "2024.04.23/"
mymap = plt.get_cmap("Spectral")
Forces = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
n = len(Forces)

plt.figure(f'Anche {j+1}')
plt.title(f'', fontsize=16)

for j in range (n):
    # recuperation donnees
    data = pd.read_table(dossier + f'anche{j+1}/data_anche{j+1}.txt', sep=' ', header=0).values 
    indices_tries = np.lexsort((data[:, 1], data[:, 0]))
    data = data[indices_tries] 
    y = data[:,0]
    z = data[:,1]
    force = data[:,2]
    N = 6
    couleur = mymap(j / n)
    ys = []
    pentes = []
    
    for i in range (0, len(data), N):
        # extractions des paramètres de la courbe paramétrique
        coefficients = np.polyfit(z[i+1:i+N], force[i+1:i+N], 1)
        pente = coefficients[0]
        pente = round(pente, 2)
        ys.append(y[i])
        pentes.append(pente)
        # pentes.append(['anche', j, 'position y', y[i], 'pente', pente])
        
    plt.plot(ys, pentes, '.-', color=couleur, label=f'{Forces[j]}')
        
        # trace la courbe paramétrique sur la section qui nous intéresse
        # plt.plot(x=data_a[i:i+4,1], np.polyval(coefficients, data_a[i:i+4,1]), label=f'Droite de régression (pente={pente:.2f}, ordonnée origine={ordonnee_origine:.2f})')
        # plt.plot(z[i:i+N], np.polyval(coefficients, z[i:i+N]), linestyle='--', color=couleur, label=f'pente={pente:.2f}')
        # plt.errorbar(z[i:i+N], force[i:i+N], yerr=ecart_type, fmt='.', color=couleur, capsize=5)

    plt.xlabel('Coordonnée y (mm)', fontsize=16)
    plt.ylabel('Raideur $k=F/z$ (N/mm)', fontsize=16)
    plt.grid(True)
    plt.legend(title='Anche')
        
    
# print(f'pente n°{i} = {pentes} N/mm')
    # enregister en .txt
    # np.savetxt(dossier + savename, 'pentes')
print(len(pentes))
print(len(ys))
plt.show()

9
9


## Traitement images

In [14]:
# Charger l'image
# dossier = "Banc statique/2024.04.05/anche1/"
dossier = "2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
# angle_rot = im_rotate(image)
# print(angle)

In [15]:
plt.close('all')

# Charger l'image
dossier = "Banc statique/2024.04.23/"
nom_anche = 'anche1/'
image = cv2.imread(dossier + nom_anche + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
angle_rot = im_rotate(image)
angle_rot = angle_rot[0]
# print(angle_rot)

# repère des bords de l'anche
cibles = ['Anche Gauche', 'Anche Droite']
hmax = 350  # hauteur de l'image

for R in range (9):
    for r in range (6):
        I = cv2.imread(dossier + nom_anche + f'image_{R}_{r}.jpg',  cv2.IMREAD_GRAYSCALE)
        # I_rotated = rotate(I, np.rad2deg(-angle_rot), mode='nearest')
        I = np.flipud(I) # Reverse the order of elements along axis 0
        I = I[150:hmax, :]  # couper l'image pour ne garder qu'une partie utile
        print(I)
        
        W = I.shape[1]  # Width
        H = I.shape[0]  # Height
        
        X = np.arange(1,W+1) 
        Y = np.arange(1,H+1)
        a = 30
        N = 20  # taille fenêtre glissante (en px)
        dist_px = 5

        # # Trouver les bords de l'anche, on gardera les indices pour toutes les anches mesurées de la série
        Ncibles = len(cibles)
        COLORS = plt.cm.cool(np.linspace(0, 1, Ncibles))

        Xcibles = np.full(Ncibles, np.nan)
        Ycibles = np.full(Ncibles, np.nan)

NameError: name 'im_rotate' is not defined

In [ ]:
import cv2
import numpy as np

# Charger les images
image = cv2.imread(dossier + nom_anche + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
clone = image.copy()

# Définir une liste pour stocker les coordonnées des points
points = []
# Définir les coordonnées du milieu à l'extérieur de la fonction click_callback
milieu_x = None
milieu_y = None

# Définir une fonction de rappel pour la souris
def click_callback(event, x, y, flags, param):
    # Vérifier si un clic gauche a été effectué
    if event == cv2.EVENT_LBUTTONDOWN:
        # Ajouter les coordonnées du point à la liste
        points.append((x, y))
        # Dessiner un cercle sur l'image pour indiquer le point
        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        # Afficher l'image mise à jour
        cv2.imshow("Image", clone)

        # Si deux points ont été sélectionnés, calculer le milieu
        if len(points) == 2:
            milieu_x = (points[0][0] + points[1][0]) // 2
            milieu_y = (points[0][1] + points[1][1]) // 2
            # Dessiner un marqueur pour le milieu
            cv2.circle(clone, (milieu_x, milieu_y), 5, (0, 255, 0), -1)
            # Afficher l'image mise à jour
            cv2.imshow("Image", clone)

            # Afficher les coordonnées du milieu
            print("Coordonnées du milieu :")
            print(f"x = {milieu_x}, y = {milieu_y}")

            return milieu_x, milieu_y
            
            
# Créer une fenêtre pour afficher l'image
cv2.namedWindow("Image")
# Attacher la fonction de rappel à la fenêtre
cv2.setMouseCallback("Image", click_callback)

# Afficher l'image
cv2.imshow("Image", clone)
# Attendre que l'utilisateur appuie sur une touche
cv2.waitKey(0)
cv2.destroyAllWindows()

# Afficher les coordonnées des deux points
print("Coordonnées des points sélectionnés :")
for i, point in enumerate(points):
    print(f"Point {i+1}: {point}")
    
    

Coordonnées du milieu :
x = 1303, y = 564
Coordonnées des points sélectionnés :
Point 1: (1255, 564)
Point 2: (1352, 564)


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [ ]:
for R in range (9):
    for r in range (6):
        I = cv2.imread(dossier + nom_anche + f'image_{R}_{r}.jpg',  cv2.IMREAD_GRAYSCALE)
        
        # Calculer le déplacement du point d'intérêt par rapport à l'image d'origine
        deplacement_x = milieu_x - 100  # (par exemple, point d'intérêt original à la coordonnée x = 100)
        deplacement_y = milieu_y - 200  # (par exemple, point d'intérêt original à la coordonnée y = 200)

        # Nouvelles coordonnées du point d'intérêt sur l'image actuelle
        nouveau_point_x = milieu_x + deplacement_x
        nouveau_point_y = milieu_y + deplacement_y

        # Dessiner un marqueur pour le point d'intérêt sur l'image actuelle
        image_actuelle_coloree = cv2.cvtColor(I, cv2.COLOR_GRAY2BGR)
        cv2.circle(image_actuelle_coloree, (nouveau_point_x, nouveau_point_y), 5, (0, 255, 0), -1)

    # Afficher l'image avec le marqueur du point d'intérêt
    cv2.imshow(f"Image {i}", image_actuelle_coloree)

    # Attendre jusqu'à ce qu'une touche soit pressée
    cv2.waitKey(0)
    cv2.destroyAllWindows()

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [ ]:
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
# contours = cv2.Canny(image_blurred, 50, 150)  # Les valeurs 50 et 150 sont des seuils min et max

# Appliquer un seuillage à l'image (si nécessaire)
_, thresholded_image = cv2.threshold(image_grayscale, 127, 255, cv2.THRESH_BINARY)

# Trouver les contours dans l'image seuillée
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dessiner les contours trouvés sur l'image d'origine
contoured_image = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
cv2.imshow('Contours', contoured_image)
# plt.imshow(contours, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()


NameError: name 'cv2' is not defined

In [ ]:
plt.close('all')
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer le seuillage pour obtenir une image binaire (noir et blanc)
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
contours = cv2.Canny(image_blurred, 30, 90)  # Les valeurs 50 et 150 sont des seuils min et max

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(binary_image, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()
